# DSCI 100 Project - Section 6 Group 16 
# Minecraft project report 
## members:

|Name|Student Number|
|--------|---------|
|Clement Cheung|40162752|
|Eileen Limneus|81033300|
|Min Hyung Yoo|26944660|
|Tyler Lee|12253688|


### Project Objectives!
+ **Broad Question:** (1) What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

+ **Specific Question:** How does a player's tendency to revisit the PLAICraft server, and their total playtime affect their subscription status to a game-related newsletter?

In [1]:
# loading the libraries

## Introduction

+ relevant background information on the topic enough so that somoneone that is unfamiliar on it can understand the full report.
+ clearly state the question you tried to answer with your project
+ identify and fully describe the dataset that was used to answer the question

## Methods & Results

* describe the methods you used to perform your analysis from beginning to end that narrates the analysis code.
* your report should include code which:
    - loads data 
    - wrangles and cleans the data to the format necessary for the planned analysis
    - performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
    - creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
    - performs the data analysis
    - creates a visualization of the analysis
 
note: all figures should have a figure number and a legend

## Discussion

* summary of findings
* discuss if results were expected
* impact of the findings
* further questions to ask

## References

* any format so long as its a consistent citation style